In [15]:
from __future__ import print_function

import logging
#logging.basicConfig(level=logging.INFO,format="%(filename)s[line:%(lineno)d] %(levelname)s %(message)s")
#logger=logging.getLogger(__name__)



import sys
sys.path.append("..")

import torch
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data.dataloader as Data


from adversary import Adversary
from gradient_method import FGSM
from pytorch import PytorchModel
#from tutorials.mnist_model_pytorch import Net
import torch.nn as nn

In [8]:
#coding=utf-8

# Copyright 2017 - 2018 Baidu Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
CNN on mnist data using pytorch
"""
from __future__ import print_function
from __future__ import division

from builtins import range
from past.utils import old_div
import os
import torch
import torchvision
from torch.autograd import Variable
import torch.utils.data.dataloader as Data

#查看cuda版本
#cat /usr/local/cuda/version.txt
#当cuda版本为8时
#pip install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp27-cp27mu-linux_x86_64.whl
#pip install torchvision
#


train_data = torchvision.datasets.MNIST(
 './mnist-pytorch/data', train=True, transform=torchvision.transforms.ToTensor(), download=True
)
test_data = torchvision.datasets.MNIST(
 './mnist-pytorch/data', train=False, transform=torchvision.transforms.ToTensor()
)
print("train_data:", train_data.train_data.size())
print("train_labels:", train_data.train_labels.size())
print("test_data:", test_data.test_data.size())


#批大小
batch_size=128
#训练的批次数
epochs=10

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, 3, 1, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, 3, 1, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 64, 3, 1, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(64 * 3 * 3, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 10)
        )


    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(conv1_out)
        conv3_out = self.conv3(conv2_out)
        res = conv3_out.view(conv3_out.size(0), -1)
        out = self.dense(res)
        #return torch.nn.functional.log_softmax(out, dim=1)
        return out


train_data: torch.Size([60000, 28, 28])
train_labels: torch.Size([60000])
test_data: torch.Size([10000, 28, 28])


C:\Users\hp\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
C:\Users\hp\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:52: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
C:\Users\hp\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [16]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels,kernel_size
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

In [2]:
def train():

    # 自适应使用GPU还是CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = Net().to(device)

    optimizer = torch.optim.Adam(model.parameters())
    loss_func = torch.nn.CrossEntropyLoss()


    train_loader = Data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    test_loader = Data.DataLoader(dataset=test_data, batch_size=batch_size)

    for epoch in range(epochs):
        sum_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # 梯度清零
            optimizer.zero_grad()

            # forward + backward
            outputs = model(inputs)

            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

            # 每训练100个batch打印一次平均loss
            sum_loss += loss.item()
            if i % 50 == 0:
                print('epoch=%d, batch=%d loss: %.04f'
                      % (epoch + 1, i, old_div(sum_loss, 100)))
                sum_loss = 0.0
        # 每跑完一次epoch测试一下准确率 进入测试模式 禁止梯度传递
        with torch.no_grad():
            correct = 0
            total = 0
            for data in test_loader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                # 取得分最高的那个类
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()
            print('epoch=%d accuracy=%.02f%%' % (epoch + 1, (old_div(100 * correct, total))))

    torch.save(model.state_dict(), 'net.pth')


train()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist-pytorch/data\MNIST\raw\train-images-idx3-ubyte.gz to ./mnist-pytorch/data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist-pytorch/data\MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist-pytorch/data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist-pytorch/data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist-pytorch/data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist-pytorch/data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist-pytorch/data\MNIST\raw

train_data: torch.Size([60000, 28, 28])
train_labels: torch.Size([60000])
test_data: torch.Size([10000, 28, 28])


C:\Users\26357\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
C:\Users\26357\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:52: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
C:\Users\26357\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


epoch=1, batch=0 loss: 0.0230
epoch=1, batch=50 loss: 0.6258
epoch=1, batch=100 loss: 0.1662
epoch=1, batch=150 loss: 0.1117
epoch=1, batch=200 loss: 0.0915
epoch=1, batch=250 loss: 0.0668
epoch=1, batch=300 loss: 0.0590
epoch=1, batch=350 loss: 0.0641
epoch=1, batch=400 loss: 0.0474
epoch=1, batch=450 loss: 0.0449
epoch=1 accuracy=97.98%
epoch=2, batch=0 loss: 0.0007
epoch=2, batch=50 loss: 0.0393
epoch=2, batch=100 loss: 0.0364
epoch=2, batch=150 loss: 0.0355
epoch=2, batch=200 loss: 0.0360
epoch=2, batch=250 loss: 0.0362
epoch=2, batch=300 loss: 0.0309
epoch=2, batch=350 loss: 0.0319
epoch=2, batch=400 loss: 0.0255
epoch=2, batch=450 loss: 0.0337
epoch=2 accuracy=98.50%
epoch=3, batch=0 loss: 0.0008
epoch=3, batch=50 loss: 0.0263
epoch=3, batch=100 loss: 0.0251
epoch=3, batch=150 loss: 0.0262
epoch=3, batch=200 loss: 0.0238
epoch=3, batch=250 loss: 0.0227
epoch=3, batch=300 loss: 0.0202
epoch=3, batch=350 loss: 0.0240
epoch=3, batch=400 loss: 0.0197
epoch=3, batch=450 loss: 0.0166
e

In [27]:
def main():
    """
    Advbox demo which demonstrate how to use advbox.
    """
    TOTAL_NUM = 500
##***********************************************    
    pretrained_model='lenetc.pth'
##***********************************************    

    loss_func = torch.nn.CrossEntropyLoss()

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./mnist-pytorch/data', train=False, 
                       download=True, transform=transforms.Compose([
            transforms.ToTensor(),
        ])),
        batch_size=1, shuffle=True)

    # Define what device we are using
    logging.info("CUDA Available: {}".format(torch.cuda.is_available()))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize the network
##***********************************************    
    model = LeNet().to(device)
##*************************************************
    # Load the pretrained model
    
    model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))
    
    # Set the model in evaluation mode. In this case this is for the Dropout layers
    model.eval()

    # advbox demo
    m = PytorchModel(model, loss_func,(0, 1),channel_axis=1)
    
    attack = FGSM(m)

    attack_config = {"epsilons": 0.3}

    # use test data to generate adversarial examples
    total_count = 0
    fooling_count = 0
    for i, data in enumerate(test_loader):
        inputs, labels = data

        #inputs, labels = inputs.to(device), labels.to(device)
        inputs, labels=inputs.numpy(),labels.numpy()

        #inputs.requires_grad = True
        #print(inputs.shape)

        total_count += 1
        adversary = Adversary(inputs, labels[0])

        # FGSM non-targeted attack
        adversary = attack(adversary, **attack_config)


        if adversary.is_successful():
            fooling_count += 1
            print(
                'attack success, original_label=%d, adversarial_label=%d, count=%d'
                % (labels, adversary.adversarial_label, total_count))

        else:
            print('attack failed, original_label=%d, count=%d' %
                  (labels, total_count))

        if total_count >= TOTAL_NUM:
            print(
                "[TEST_DATASET]: fooling_count=%d, total_count=%d, fooling_rate=%f"
                % (fooling_count, total_count,
                   float(fooling_count) / total_count))
            break
    print("fgsm attack done")


if __name__ == '__main__':
    main()

cpu
attack success, original_label=1, adversarial_label=3, count=1
attack success, original_label=2, adversarial_label=1, count=2
attack success, original_label=8, adversarial_label=2, count=3
attack success, original_label=0, adversarial_label=6, count=4
attack success, original_label=6, adversarial_label=3, count=5
attack success, original_label=7, adversarial_label=3, count=6
attack success, original_label=9, adversarial_label=7, count=7
attack success, original_label=3, adversarial_label=7, count=8
attack success, original_label=4, adversarial_label=2, count=9
attack success, original_label=1, adversarial_label=3, count=10
attack success, original_label=7, adversarial_label=3, count=11
attack success, original_label=3, adversarial_label=5, count=12
attack success, original_label=5, adversarial_label=3, count=13
attack success, original_label=6, adversarial_label=0, count=14
attack success, original_label=6, adversarial_label=2, count=15
attack success, original_label=5, adversarial

attack success, original_label=1, adversarial_label=2, count=134
attack success, original_label=3, adversarial_label=5, count=135
attack success, original_label=0, adversarial_label=7, count=136
attack success, original_label=2, adversarial_label=8, count=137
attack success, original_label=6, adversarial_label=7, count=138
attack success, original_label=7, adversarial_label=8, count=139
attack success, original_label=3, adversarial_label=2, count=140
attack success, original_label=9, adversarial_label=3, count=141
attack success, original_label=2, adversarial_label=3, count=142
attack success, original_label=3, adversarial_label=2, count=143
attack success, original_label=2, adversarial_label=1, count=144
attack success, original_label=9, adversarial_label=4, count=145
attack success, original_label=2, adversarial_label=3, count=146
attack success, original_label=9, adversarial_label=8, count=147
attack success, original_label=9, adversarial_label=2, count=148
attack success, original_

attack success, original_label=3, adversarial_label=2, count=273
attack success, original_label=1, adversarial_label=2, count=274
attack success, original_label=6, adversarial_label=3, count=275
attack success, original_label=4, adversarial_label=8, count=276
attack success, original_label=0, adversarial_label=3, count=277
attack success, original_label=7, adversarial_label=3, count=278
attack success, original_label=2, adversarial_label=5, count=279
attack success, original_label=0, adversarial_label=2, count=280
attack success, original_label=6, adversarial_label=9, count=281
attack success, original_label=7, adversarial_label=3, count=282
attack success, original_label=9, adversarial_label=4, count=283
attack success, original_label=7, adversarial_label=3, count=284
attack success, original_label=6, adversarial_label=5, count=285
attack success, original_label=1, adversarial_label=7, count=286
attack success, original_label=5, adversarial_label=2, count=287
attack success, original_

attack success, original_label=9, adversarial_label=2, count=414
attack success, original_label=9, adversarial_label=4, count=415
attack success, original_label=5, adversarial_label=3, count=416
attack success, original_label=7, adversarial_label=3, count=417
attack success, original_label=3, adversarial_label=5, count=418
attack success, original_label=2, adversarial_label=3, count=419
attack success, original_label=2, adversarial_label=3, count=420
attack success, original_label=7, adversarial_label=3, count=421
attack success, original_label=3, adversarial_label=5, count=422
attack success, original_label=8, adversarial_label=3, count=423
attack success, original_label=5, adversarial_label=3, count=424
attack success, original_label=1, adversarial_label=8, count=425
attack success, original_label=2, adversarial_label=7, count=426
attack success, original_label=9, adversarial_label=7, count=427
attack success, original_label=7, adversarial_label=8, count=428
attack success, original_

In [29]:
def train():

    # 自适应使用GPU还是CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model =LeNet().to(device)

    optimizer = torch.optim.Adam(model.parameters())
    loss_func = torch.nn.CrossEntropyLoss()


    train_loader = Data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    test_loader = Data.DataLoader(dataset=test_data, batch_size=batch_size)
##***************************************
    for epoch in range(epochs):
##***************************************
        sum_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # 梯度清零
            optimizer.zero_grad()

            # forward + backward
            outputs = model(inputs)

            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

            # 每训练100个batch打印一次平均loss
            sum_loss += loss.item()
            if i % 50 == 0:
                print('epoch=%d, batch=%d loss: %.04f'
                      % (epoch + 1, i, old_div(sum_loss, 100)))
                sum_loss = 0.0
        # 每跑完一次epoch测试一下准确率 进入测试模式 禁止梯度传递
        with torch.no_grad():
            correct = 0
            total = 0
            for data in test_loader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                # 取得分最高的那个类
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()
            print('epoch=%d accuracy=%.02f%%' % (epoch + 1, (old_div(100 * correct, total))))

    torch.save(model.state_dict(), 'lenetc.pth')


train()

epoch=1, batch=0 loss: 0.0238
epoch=1, batch=50 loss: 1.1536
epoch=1, batch=100 loss: 1.1518
epoch=1, batch=150 loss: 1.1479
epoch=1, batch=200 loss: 1.0564
epoch=1, batch=250 loss: 0.7513
epoch=1, batch=300 loss: 0.5092
epoch=1, batch=350 loss: 0.3817
epoch=1, batch=400 loss: 0.3111
epoch=1, batch=450 loss: 0.2635
epoch=1 accuracy=89.14%
epoch=2, batch=0 loss: 0.0038
epoch=2, batch=50 loss: 0.2132
epoch=2, batch=100 loss: 0.1886
epoch=2, batch=150 loss: 0.1657
epoch=2, batch=200 loss: 0.1456
epoch=2, batch=250 loss: 0.1542
epoch=2, batch=300 loss: 0.1332
epoch=2, batch=350 loss: 0.1239
epoch=2, batch=400 loss: 0.1183
epoch=2, batch=450 loss: 0.1176
epoch=2 accuracy=94.24%
epoch=3, batch=0 loss: 0.0022
epoch=3, batch=50 loss: 0.1048
epoch=3, batch=100 loss: 0.0990
epoch=3, batch=150 loss: 0.0911
epoch=3, batch=200 loss: 0.0891
epoch=3, batch=250 loss: 0.0852
epoch=3, batch=300 loss: 0.0915
epoch=3, batch=350 loss: 0.0846
epoch=3, batch=400 loss: 0.0829
epoch=3, batch=450 loss: 0.0776
e